# Advanced Applications of Mutate

## Three helpful functions

In [1]:
# Install stuff
!pip install unpythonic

     |████████████████████████████████| 238 kB 2.7 MB/s eta 0:00:01


In [1]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [3]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Exhibitions

In [2]:
exhib_url = "https://github.com/MuseumofModernArt/exhibitions/raw/master/MoMAExhibitions1929to1989.csv"
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate', 'ConstituentBeginDate' ,'ConstituentEndDate']
exhibitions = pd.read_csv(exhib_url, 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(2)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902,1981,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839,1906,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053


#### MoMA Artists

In [3]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


#### MoMA Artwork

In [4]:
from more_dfply import fix_names

artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


# Three helpful column functions

In this section, we will focus on two useful column functions: `ifelse`, `coalesce` and `case_when`

# Branching with `ifelse`

The functions `ifelse` 

* allows us to pick between two options in a `mutate`.
* has the following syntax: `ifelse(cond, then, else)`
* Will return `then` with `cond == True`
* Will return `else` with `cond == False`

In [8]:
from more_dfply import ifelse

(artwork
 >> select(X.Gender)
 >> mutate(recode_gender = ifelse(X.Gender == '(Male)',
                                  'm',
                                  'f'))
 >>head
)

,Gender,recode_gender
0,(Male),m
1,(Male),m
2,(Male),m
3,(Male),m
4,(Male),m


### `then` and `else` conform to the `len(cond)`

* Singletons are repeated.
* Short vectors are tiled.
* Series/lists that are too long are truncated.

#### Some example conditions

In [9]:
from numpy import repeat, arange
all_true = repeat(True, 5)
all_false = repeat(False, 5)
all_true, all_false

(array([ True,  True,  True,  True,  True]),
 array([False, False, False, False, False]))

#### Series that are too long or too short

In [9]:
short = arange(1,4,1)
long = arange(1,10,1)
short, long

(array([1, 2, 3]), array([1, 2, 3, 4, 5, 6, 7, 8, 9]))

#### Singletons are repeated

In [10]:
ifelse(all_true, 'singleton', long)

0    singleton
1    singleton
2    singleton
3    singleton
4    singleton
dtype: object

#### Short sequences are tiled

The sequence of a short sequence is repeated, over and over, until it has the same length as `cond`

In [11]:
ifelse(all_true, short, long)

0    1
1    2
2    3
3    1
4    2
dtype: int64

#### Long sequences are truncated

The sequence of a long sequence is repeated, over and over, until it has the same length as `cond`

In [12]:
ifelse(all_false, short, long)

0    1
1    2
2    3
3    4
4    5
dtype: int64

### `then` and `else` are only evaluated if needed (when `Intention`s)

* If `cond` is all true, then `else Intention` will not be evaluated.
* Similarly, if `cond` is all false, the an `then Intention` will not be evaluated.

#### It is important that conditionals don't evaluate the "other" expression

In [13]:
'safe' if True else 1/0

'safe'

In [6]:
def my_ifelse(cond, then, else_):
    return then if cond else else_

In [15]:
my_ifelse(True, 'safe', 1/0)

ZeroDivisionError: division by zero

#### An expression that will crash if `else` is evaluated

In [16]:
(X/0).evaluate(2)

ZeroDivisionError: division by zero

#### No crash $\Rightarrow$ `else` was not evaluated

In [17]:
ifelse(all_true, 'safe', X.height_cm/0)

In [18]:
ifelse(all_true, 'safe', X.height_cm/0).evaluate(2)

0    safe
1    safe
2    safe
3    safe
4    safe
dtype: object

## <font color="red"> Exercise 3.3.1 </font>

Consider the `Nationality` column `exhibition` table.  We would like to make a new column that reclassifies this column titled `"American"` that contains `1` if the artist is of American decent and `0` otherwise. 

In [10]:
(exhibitions
 >> mutate(American = ifelse(X.Nationality=='American', 1,0))
 
 >> select(X.Nationality, X.American)
 >> head(10)
)

,Nationality,American
0,American,1
1,French,0
2,French,0
3,Dutch,0
4,French,0
5,American,1
6,American,1
7,American,1
8,American,1
9,American,1


## Generalizing `ifelse` with `case_when`

`case_when` takes one more `(pred, then)` tuples
* `pred` is a `bool` expression
* `then` is added/coalesced with the answer when `pred == True`

This is similar to the R `case_when` from `dplyr`. See [case_when docs](https://dplyr.tidyverse.org/reference/case_when.html)

In [25]:
from more_dfply import case_when

#### Some example conditions

In [26]:
df = pd.DataFrame({'cat':['a','b','b','c','c'],
                   'val':[ 1,  1,  2,  1, 2]})
df

,cat,val
0,a,1
1,b,1
2,b,2
3,c,1
4,c,2


#### `case_when` with one predicate pair

Unmatched values are `nan`

In [27]:
(df
 >> mutate(new = case_when((df.cat == 'a', df.val + 1))))

,cat,val,new
0,a,1,2.0
1,b,1,NaN
2,b,2,NaN
3,c,1,NaN
4,c,2,NaN


#### Left-hand pairs have precident

In [28]:
(df
 >> mutate(new = case_when((df.cat == 'a', df.val + 1),
                           (df.cat == 'b', df.val + 2))))

,cat,val,new
0,a,1,2.0
1,b,1,3.0
2,b,2,4.0
3,c,1,NaN
4,c,2,NaN


#### Singletons are accepted

In [29]:
(df
 >> mutate(new =  case_when((df.cat == 'a', df.val + 1),
                            (df.cat == 'b', df.val + 2),
                            (df.cat == 'c', 18))))

,cat,val,new
0,a,1,2.0
1,b,1,3.0
2,b,2,4.0
3,c,1,18.0
4,c,2,18.0


#### `case_when` handles the `X Intention`

In [30]:
(df
 >> mutate(new =  case_when((X.cat == 'a', X.val + 1),
                            (X.cat == 'b', X.val + 2),
                            (X.cat == 'c', 18))))

,cat,val,new
0,a,1,2.0
1,b,1,3.0
2,b,2,4.0
3,c,1,18.0
4,c,2,18.0


## <font color="red"> Exercise 3.3.2 </font>

Consider the `Nationality` column `exhibition` table.  We would like to make a new column that reclassifies this column as `"North American"`, `"European"`, or `"Other"`.  Use `case_when` to accomplish this task. 

In [31]:
exhibitions.Nationality.unique()

array(['American', 'French', 'Dutch', 'Italian', nan, 'Spanish', 'German',
       'Mexican', 'Austrian', 'Finnish', 'Swedish', 'architect', 'Swiss',
       'British', 'Czech', 'Belgian', 'Russian', 'Guatemalan',
       'Russian-Lithuanian', 'English', 'Nationality unknown', 'Greek',
       'Norwegian', 'Georgian', 'Latvian', 'Polish', 'Japanese',
       'Milanese', 'Danish', 'Netherlandish', 'Romanian', 'Flemish',
       'Israeli', 'Scottish', 'Hungarian', 'Yugoslav', 'Brazilian',
       'Ukrainian', 'Catalan', 'Florentine', 'Venetian', 'Peruvian',
       'Canadian', 'Bolivian', 'Cuban', 'Irish', 'Chinese', 'Argentine',
       'Chilean', 'Colombian', 'Uruguayan', 'Ecuadorian', 'Venezuelan',
       'Australian', 'Haitian', 'Indian', 'Korean', 'Turkish', 'New',
       'Tanzanian', 'New Zealander', 'South', 'Icelandic', 'Iranian',
       'Panamanian', 'Rhodesian', 'Sudanese', 'Moroccan and American',
       'Canadian Inuit', 'Slovene', 'Bosnian', 'South African',
       'Croatian', 'Luxem

In [60]:
(exhibitions
>> mutate(region = case_when((X.Nationality == 'American', 'North American'),
                             (X.Nationality == 'Mexican', 'North American'),
                             (X.Nationality == 'Guatemalan', 'North American'),
                             (X.Nationality == 'Canadian', 'North American'),
                             (X.Nationality == 'Cuban', 'North American'),
                             (X.Nationality == 'Haitian', 'North American'),
                             (X.Nationality == 'Panamanian', 'North American'),
                             (X.Nationality == 'Candian Inuit', 'North American'),
                             (X.Nationality == 'Native American', 'North American'),
                             (X.Nationality == 'American and Mexican', 'North American'),
                             
                             (X.Nationality == 'French', 'European'),
                             (X.Nationality == 'Dutch', 'European'),
                             (X.Nationality == 'Italian', 'European'),
                             (X.Nationality == 'Spanish', 'European'),
                             (X.Nationality == 'German', 'European'),
                             (X.Nationality == 'Austrian', 'European'),
                             (X.Nationality == 'Finnish', 'European'),
                             (X.Nationality == 'Swedish', 'European'),
                             (X.Nationality == 'Swiss', 'European'),
                             (X.Nationality == 'British', 'European'),
                             (X.Nationality == 'Czech', 'European'),
                             (X.Nationality == 'Belgian', 'European'),
                             (X.Nationality == 'Russian', 'European'),
                             (X.Nationality == 'Russian-Lithuanian', 'European'),
                             (X.Nationality == 'English', 'European'),
                             (X.Nationality == 'Greek', 'European'),
                             (X.Nationality == 'Norwegian', 'European'),
                             (X.Nationality == 'Georgian', 'European'),
                             (X.Nationality == 'Latvian', 'European'),
                             (X.Nationality == 'Polish', 'European'),
                             (X.Nationality == 'Milanese', 'European'),
                             (X.Nationality == 'Danish', 'European'),
                             (X.Nationality == 'Netherlandish', 'European'),
                             (X.Nationality == 'Romanian', 'European'),
                             (X.Nationality == 'Flemish', 'European'),
                             (X.Nationality == 'Scottish', 'European'),
                             (X.Nationality == 'Hungarian', 'European'),
                             (X.Nationality == 'Yugoslav', 'European'),
                             (X.Nationality == 'Ukranian', 'European'),
                             (X.Nationality == 'Catalan', 'European'),
                             (X.Nationality == 'Florentine', 'European'),
                             (X.Nationality == 'Venetian', 'European'),
                             (X.Nationality == 'Irish', 'European'),
                             (X.Nationality == 'Turkish', 'European'),
                             (X.Nationality == 'Icelandic', 'European'),
                             (X.Nationality == 'Slovene', 'European'),
                             (X.Nationality == 'Bosnian', 'European'),
                             (X.Nationality == 'Croatian', 'European'),
                             (X.Nationality == 'Luxenbourgish', 'European'),
                             
                             (X.Nationality.isnull(), 'Other'),
                             (X.Nationality == 'architect', 'Other'),
                             (X.Nationality == 'Nationality Unknown', 'Other'),
                             (X.Nationality == 'Japanese', 'Other'),
                             (X.Nationality == 'Peruvian', 'Other'),
                             (X.Nationality == 'Chinese', 'Other'),
                             (X.Nationality == 'Argentine', 'Other'),
                             (X.Nationality == 'Chilean', 'Other'),
                             (X.Nationality == 'Columbian', 'Other'),
                             (X.Nationality == 'Uruguayan', 'Other'),
                             (X.Nationality == 'Ecuadorian', 'Other'),
                             (X.Nationality == 'Venezuelan', 'Other'),
                             (X.Nationality == 'Australian', 'Other'),
                             (X.Nationality == 'Indian', 'Other'),
                             (X.Nationality == 'Korean', 'Other'),
                             (X.Nationality == 'New', 'Other'),
                             (X.Nationality == 'Tanzanian', 'Other'),
                             (X.Nationality == 'South', 'Other'),
                             (X.Nationality == 'Iranian', 'Other'),
                             (X.Nationality == 'Rhodesian', 'Other'),
                             (X.Nationality == 'Sudanese', 'Other'),
                             (X.Nationality == 'Moroccan and American', 'Other'),
                             (X.Nationality == 'South African', 'Other')
                        )
            )
 
 >> tail(30)
)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL,region
34528,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,American,1912,1956,"American, 19121956",Male,12316903.0,Q37571,500015134.0,moma.org/artists/4675,North American
34529,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,German,1849,1938,"German, 18491938",Male,36949603.0,Q62992,500017013.0,moma.org/artists/5000,European
34530,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,French,1871,1958,"French, 18711958",Male,16857.0,Q156128,500024798.0,moma.org/artists/5053,European
34531,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,French,1864,1901,"French, 18641901",Male,32003649.0,Q82445,500029114.0,moma.org/artists/5910,European
34532,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,French,1875,1963,"French, 18751963",Male,32004174.0,Q452254,500003616.0,moma.org/artists/6165,European
34533,392.0,1535,Prints: Proofs and Variants,"[MoMA Exh. #1535, November 16, 1989-March 13, ...",1989-11-16,1990-03-13,1765.0,moma.org/calendar/exhibitions/1740,Artist,Artist,...,American,1881,1961,"American, born Russia. 18811961",Male,41883358.0,Q535334,500029261.0,moma.org/artists/6275,North American
34534,NaN,No#,Sidney Janis Memorial,"[November 30, 1989-January 3, 1990]",1989-11-30,1990-01-03,1766.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
34535,288.0,1536,Recent Japanese Posters from the Collection,"[MoMA Exh. #1536, December 9, 1989-April 16, 1...",1989-12-09,1990-04-16,1767.0,moma.org/calendar/exhibitions/1739,Curator,Organizer,...,NaN,NaN,NaN,NaN,NaN,92938971.0,NaN,NaN,moma.org/artists/10707,Other
34536,288.0,1536,Recent Japanese Posters from the Collection,"[MoMA Exh. #1536, December 9, 1989-April 16, 1...",1989-12-09,1990-04-16,1767.0,moma.org/calendar/exhibitions/1739,Artist,Artist,...,Japanese,1929,2009,"Japanese, 19292009",Male,96609855.0,NaN,NaN,moma.org/artists/254,Other
34537,288.0,1536,Recent Japanese Posters from the Collection,"[MoMA Exh. #1536, December 9, 1989-April 16, 1...",1989-12-09,1990-04-16,1767.0,moma.org/calendar/exhibitions/1739,Artist,Artist,...,Japanese,1932,2009,"Japanese, 19322009",Male,NaN,NaN,NaN,moma.org/artists/2026,Other


In [40]:
(exhibitions
>> mutate(region = case_when((X.Nationality == 'American' or
                             X.Nationality == 'Mexican','North American')) )
 >> head(20)
)

TypeError: __index__ returned non-int (type Intention)

# Using `coalesce` to remove missing values

* Syntax: `coalesce(col1, col2, ...)`
* Returns a `pd.Series`
* Each entry is the first non-missing value from `col1`, `col2`, ... (working left to right).

In [61]:
from more_dfply import coalesce

In [62]:
df = pd.DataFrame({'cat':['a','b','b','c','c'],
                   'val':[ 1,  1,  2,  1, 2]})
df

,cat,val
0,a,1
1,b,1
2,b,2
3,c,1
4,c,2


#### Example `df` with some missing values

In [63]:
df = pd.DataFrame(np.random.randint(0, 10, size=(5, 3)), columns=list('abc'))
df.loc[::2, 'a'] = np.nan
df.loc[::3, 'b'] = np.nan
df

,a,b,c
0,NaN,NaN,2
1,4.0,5.0,9
2,NaN,5.0,2
3,9.0,NaN,2
4,NaN,6.0,9


#### `coaleace` first two columns

In [64]:
(df
>> mutate(d = coalesce(df.a, df.b)))

,a,b,c,d
0,NaN,NaN,2,NaN
1,4.0,5.0,9,4.0
2,NaN,5.0,2,5.0
3,9.0,NaN,2,9.0
4,NaN,6.0,9,6.0


#### `coaleace` first all three columns

In [65]:
(df
>> mutate(d = coalesce(df.a, df.b, df.c)))

,a,b,c,d
0,NaN,NaN,2,2.0
1,4.0,5.0,9,4.0
2,NaN,5.0,2,5.0
3,9.0,NaN,2,9.0
4,NaN,6.0,9,6.0


#### `coaleace` handles `dfply.Intention`s

In [66]:
(df
>> mutate(d = coalesce(X.a, X.b)))

,a,b,c,d
0,NaN,NaN,2,NaN
1,4.0,5.0,9,4.0
2,NaN,5.0,2,5.0
3,9.0,NaN,2,9.0
4,NaN,6.0,9,6.0


#### `coalesce` ignores unnecessary arguments

In [67]:
(df
 >> mutate(d = coalesce(X.a, df.b, df.c, 
                        X.d, X/0 # Ignored
                       )))

,a,b,c,d
0,NaN,NaN,2,2.0
1,4.0,5.0,9,4.0
2,NaN,5.0,2,5.0
3,9.0,NaN,2,9.0
4,NaN,6.0,9,6.0
